In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import json
from tqdm import tqdm

In [3]:
def create_xtest(data,n_predictions,n_next):
    """
    创建测试集
    """
    test_X = []
    for i in range(data.shape[0] - n_predictions - n_next + 1):
        a = data[(i + n_predictions - n_next ):(i +  (2 * n_predictions) - n_next),:]
        test_X.append(a)
    test_X = np.array(test_X,dtype='float64')
    return test_X
def Y_to_seq(Y):
    seq = []
    for i in range(Y.shape[0]):
        seq.append(Y[i,0])
    for i in range(1,Y.shape[1]):
        seq.append(Y[-1,i])
    return np.array(seq)
def create_dataset(data,n_predictions,n_next):
    '''
    对数据进行处理
    '''
    dim = data.shape[1]
    train_X, train_Y = [], []
    for i in range(data.shape[0]-n_predictions-n_next+1):
        a = data[i:(i+n_predictions),:]
        train_X.append(a)
        tempb = data[(i+n_predictions):(i+n_predictions+n_next),:]
        b = []
        for j in range(len(tempb)):
            for k in range(dim):
                b.append(tempb[j,k])
        train_Y.append(b)
    train_X = np.array(train_X,dtype='float64')
    train_Y = np.array(train_Y,dtype='float64')

    return train_X, train_Y
def trainModel(train_X, train_Y):
    '''
    trainX，trainY: 训练LSTM模型所需要的数据
    '''
    model = Sequential()
    model.add(LSTM(
        140,
        input_shape=(train_X.shape[1], train_X.shape[2]),
        return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(
        140,
        return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(
        train_Y.shape[1]))
    model.add(Activation("relu"))

    model.compile(loss='mse', optimizer='adam')
    model.fit(train_X, train_Y, epochs=100, batch_size=64, verbose=1)

    return model
def reshape_y_hat(y_hat,dim):
    re_y = np.zeros(len(y_hat),dtype='float64')
    length =int(len(y_hat)/dim)
    re_y = re_y.reshape(length,dim)

    for curdim in range(dim):
        for i in range(length):
            re_y[i,curdim] = y_hat[i + curdim*length]

    return  re_y

In [4]:
with open('change_flow.json','r') as f:
    dataset = json.load(f)

In [6]:
flow = {}
for i in tqdm(set(dataset)):
    flow[i] = {}
    data = pd.DataFrame(dataset[str(i)]).T
    data = data.values
    data = data.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data.ravel().reshape(-1,1)).reshape(-1,1)
    Xtrain,Ytrain = create_dataset(data,48,24)
    Xtest = create_xtest(data,48,24)
    model = trainModel(Xtrain,Ytrain)
    y_hat = model.predict(Xtest)
    Ypre = scaler.inverse_transform(y_hat)
    Y_output = Y_to_seq(Ypre)
    y = Y_output[-24:]
    for j in range(24):
        flow[i][j+1] = y[j]

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
1345/1345 [==============================] - 4s 3ms/sample - loss: 0.0832
Epoch 2/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0484
Epoch 3/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0456
Epoch 4/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0407
Epoch 5/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0388
Epoch 6/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0368
Epoch 7/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0350
Epoch 8/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0314
Epoch 9/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0299
Epoch 10/100
1

  9%|███████▍                                                                          | 1/11 [04:25<44:19, 265.95s/it]

Epoch 1/100
1345/1345 [==============================] - 4s 3ms/sample - loss: 0.0563
Epoch 2/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0373
Epoch 3/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0353
Epoch 4/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0331
Epoch 5/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0285
Epoch 6/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0272
Epoch 7/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0267
Epoch 8/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0266
Epoch 9/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0263
Epoch 10/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0260
Epoch 11/100
1345/1345 [==============================] - 3s 2ms/sample - loss: 0.0255
Epoch 12/100
1345/1345 [============================

 18%|██████████████▉                                                                   | 2/11 [09:14<41:51, 279.02s/it]

...

 82%|█████████████████████████████████████████████████████████████████▍              | 9/11 [1:19:37<22:35, 677.85s/it]

Epoch 1/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.1433
Epoch 2/100
1345/1345 [==============================] - 9s 7ms/sample - loss: 0.0757
Epoch 3/100
1345/1345 [==============================] - 9s 6ms/sample - loss: 0.0618
Epoch 4/100
1345/1345 [==============================] - 8s 6ms/sample - loss: 0.0415
Epoch 5/100
1345/1345 [==============================] - 9s 7ms/sample - loss: 0.0313
Epoch 6/100
1345/1345 [==============================] - 9s 7ms/sample - loss: 0.0289
Epoch 7/100
1345/1345 [==============================] - 10s 7ms/sample - loss: 0.0261
Epoch 8/100
1345/1345 [==============================] - 10s 7ms/sample - loss: 0.0228
Epoch 9/100
1345/1345 [==============================] - 9s 7ms/sample - loss: 0.0181
Epoch 10/100
1345/1345 [==============================] - 10s 7ms/sample - loss: 0.0163
Epoch 11/100
1345/1345 [==============================] - 9s 7ms/sample - loss: 0.0141
Epoch 12/100
1345/1345 [========================

 91%|███████████████████████████████████████████████████████████████████████▊       | 10/11 [1:35:00<12:33, 753.63s/it]

Epoch 1/100
1345/1345 [==============================] - 13s 10ms/sample - loss: 0.0973
Epoch 2/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0525
Epoch 3/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0443
Epoch 4/100
1345/1345 [==============================] - 10s 7ms/sample - loss: 0.0287
Epoch 5/100
1345/1345 [==============================] - 11s 9ms/sample - loss: 0.0227
Epoch 6/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0211
Epoch 7/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0253
Epoch 8/100
1345/1345 [==============================] - 10s 8ms/sample - loss: 0.0207
Epoch 9/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0193
Epoch 10/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0185
Epoch 11/100
1345/1345 [==============================] - 11s 8ms/sample - loss: 0.0184
Epoch 12/100
1345/1345 [================

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [1:53:12<00:00, 617.54s/it]


In [7]:
for i in flow:
    for j in flow[i]:
        flow[i][j] = np.float64(flow[i][j])

In [8]:
with open('flow_predict.json','w') as f:
    json.dump(flow,f)